In [2]:
from dotenv import load_dotenv
import dspy
import os

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
lm = dspy.LM("openai/gpt-4.1-mini", temperature=1, api_key=api_key, max_tokens=32000)
dspy.configure(lm=lm)

In [5]:
import dspy
from datasets import load_dataset

def init_dataset():
    train_split = load_dataset("AI-MO/aimo-validation-aime")['train']
    train_split = [
        dspy.Example({
            "problem": x['problem'],
            'solution': x['solution'],
            'answer': x['answer'],
        }).with_inputs("problem")
        for x in train_split
    ]
    import random
    random.Random(0).shuffle(train_split)
    tot_num = len(train_split)

    test_split = load_dataset("MathArena/aime_2025")['train']
    test_split = [
        dspy.Example({
            "problem": x['problem'],
            'answer': x['answer'],
        }).with_inputs("problem")
        for x in test_split
    ]

    train_set = train_split[:int(0.5 * tot_num)]
    val_set = train_split[int(0.5 * tot_num):]
    test_set = test_split * 5

    return train_set, val_set, test_set

c:\Users\Gamer\Downloads\Python\ALFA UROP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_set, val_set, test_set = init_dataset()

In [ ]:
test_point = 0
print("Problem:")
print(train_set[test_point]['problem'])
print("\n\nSolution:")
print(train_set[test_point]['solution'])
print("\n\nAnswer:")
print(train_set[test_point]['answer'])

Problem:
In isosceles trapezoid $ABCD$, parallel bases $\overline{AB}$ and $\overline{CD}$ have lengths $500$ and $650$, respectively, and $AD=BC=333$. The angle bisectors of $\angle{A}$ and $\angle{D}$ meet at $P$, and the angle bisectors of $\angle{B}$ and $\angle{C}$ meet at $Q$. Find $PQ$.


Solution:
We have the following diagram:

Let $X$ and $W$ be the points where $AP$ and $BQ$ extend to meet $CD$, and $YZ$ be the height of $\triangle AZB$. As proven in Solution 2, triangles $APD$ and $DPW$ are congruent right triangles. Therefore, $AD = DW = 333$. We can apply this logic to triangles $BCQ$ and $XCQ$ as well, giving us $BC = CX = 333$. Since $CD = 650$, $XW = DW + CX - CD = 16$.
Additionally, we can see that $\triangle XZW$ is similar to $\triangle PQZ$ and $\triangle AZB$. We know that $\frac{XW}{AB} = \frac{16}{500}$. So, we can say that the height of the triangle $AZB$ is $500u$ while the height of the triangle $XZW$ is $16u$. After that, we can figure out the distance from 

In [10]:
class GenerateResponse(dspy.Signature):
    """Solve the problem and provide the answer in the correct format."""
    problem = dspy.InputField()
    answer = dspy.OutputField()

program = dspy.ChainOfThought(GenerateResponse)

In [ ]:
def metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    """
    An evaluation metric for our evaluator
    """
    correct_answer = int(example['answer'])
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        return 0
    return int(correct_answer == llm_answer)

In [12]:
import dspy
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=32,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 71.00 / 150 (47.3%): 100%|██████████| 150/150 [04:24<00:00,  1.76s/it]

2025/10/07 20:59:42 INFO dspy.evaluate.evaluate: Average Metric: 71 / 150 (47.3%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,We are given the problem of finding all integer bases \(b > 9\) fo...,70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,Let's analyze the problem step-by-step. **Given:** - On side \( AB...,588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players, each choosing a flavor from {chocolate (C), van...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We want to find integer pairs \((x,y)\) with \(-100 \leq x,y \leq ...",117,✔️ [1]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We are asked to find the number of eight-digit integers formed fro...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We are given a regular 24-gon with vertex set \( S \), and we want...",4480,
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We have an 11-sided polygon \( A_1 A_2 \ldots A_{11} \) with the f...,19,✔️ [1]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,We are given a sequence \((x_k)\) defined as: \[ x_1 = \frac{25}{1...,616,
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,Given a right triangle \(\triangle ABC\) with \(\angle A = 90^\cir...,104,✔️ [1]


EvaluationResult(score=47.33, results=<list of 150 results>)